In [ ]:
# ==========================================================
# Importando as bibliotecas necessárias
# ==========================================================
import requests
from pyspark.sql import SparkSession  # Para criar e manipular DataFrames no Spark

# Criando a sessão Spark
spark = SparkSession.builder \
    .appName("API REST Countries") \
    .getOrCreate()

print("🚀 SparkSession criada com sucesso!")

# ==========================================================
# Fazendo a requisição para a API REST Countries
# ==========================================================
url = "https://restcountries.com/v3.1/all"

# Requisição GET para obter os dados dos países
response = requests.get(url)

# Verificamos se a resposta foi bem-sucedida (código 200)
if response.status_code == 200:
    print("✅ Conexão bem-sucedida com a API REST Countries!")

    # Convertendo a resposta em formato JSON
    paises = response.json()

    # Lista para armazenar os dados dos países
    lista_paises = []

    # ==========================================================
    # Extraindo os dados desejados de cada país
    # ==========================================================
    for pais in paises:
        # Extraindo o nome comum do país
        nome = pais.get('name', {}).get('common', 'Não informado')

        # Extraindo o código alpha-2 (duas letras)
        codigo_alpha2 = pais.get('cca2', 'Não informado')

        # Extraindo o código alpha-3 (três letras)
        codigo_alpha3 = pais.get('cca3', 'Não informado')

        # Extraindo o código numérico
        codigo_numerico = pais.get('ccn3', 'Não informado')

        # Extraindo o código olímpico, se existir
        codigo_olimpico = pais.get('cioc', 'Não informado')

        # Extraindo o nome oficial do país
        official_name = pais.get("name", {}).get("official")

        # Adicionando o país na lista como dicionário
        lista_paises.append({
            'nome': nome,
            'codigo_alpha2': codigo_alpha2,
            'codigo_alpha3': codigo_alpha3,
            'codigo_numerico': codigo_numerico,
            'codigo_olimpico': codigo_olimpico,
            'official_name': official_name
        })

    print("✅ Dados dos países coletados com sucesso!")

else:
    print(f"❌ Erro na requisição. Código de status: {response.status_code}")

# ==========================================================
# Convertendo a lista para DataFrame no Databricks
# ==========================================================

# Definimos os nomes das colunas para o DataFrame
df_lista_paises = spark.createDataFrame(
    lista_paises,
    ["nome", "codigo_alpha2", "codigo_alpha3", "codigo_numerico", "codigo_olimpico","official_name"]
)

print("✅ DataFrame criado com sucesso!")

# ==========================================================
# Exibindo o DataFrame para visualização no Databricks
# ==========================================================

display(df_lista_paises)  # Comando Databricks para visualizar em formato de tabela

# ==========================================================
# Criando uma Temp View para consultas SQL no Databricks
# ==========================================================

df_lista_paises.createOrReplaceGlobalTempView("view_paises_codigos")
print("✅ Temp View 'view_paises_codigos' criada com sucesso!")
